# Import Libraries

In [7]:
import webbrowser
import glob
import pandas as pd
import time
import shutil, os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date
%matplotlib inline
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests

# Create Folders

In [8]:
today = date.today() 
try:
    os.mkdir(str(today))
    os.mkdir(str(today)+'/Patent files')
    os.mkdir(str(today)+'/Trials files')
except Exception:
    pass

# Get list of non-public companies

In [9]:
def scrape(company):
    ticker=[]
    listed=[]
    public=[]
    for i in company:
        try:
            driver.get('https://www.google.com/search?tbm=fin&q=NASDAQ%3A%20{}'.format("+".join(i.split())))
            source = driver.page_source
            soup=bs4.BeautifulSoup(source, 'html.parser')
            ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0])
            listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0])
            public.append(1)
        except:
            time.sleep(5)
            try:
                driver.get('https://www.google.com/search?tbm=fin&q={}'.format("+".join(i.split())))
                source = driver.page_source
                soup=bs4.BeautifulSoup(source, 'html.parser')
                ticker.append(soup.find('span',{'jsname':'qRSVye'}).text[0])
                listed.append(soup.find('div',{'class':'wx62f PZPZlf'}).text.split(':')[0])
                public.append(1)
            except:
                ticker.append(np.nan)
                listed.append(np.nan)
                public.append(0)
        time.sleep(5)
    return public,listed,ticker

In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # open Chrome driver/window
user=input('Do you want to enter companies one by one?(y/n): ')
l=[]
if(user=='y'):
    more='y'
    while(more=='y'):
        a=input('enter company name: ')
        l.append(a)
        more=input('do you want to enter more companies(y/n): ')
else:
    csv=input('enter name of excel file: ') # put non-public.xlsx
    companies=input('enter column containing list of companies: ') # put Company
    try:
        l=list(pd.read_csv(csv)[companies])
    except:
        l=list(pd.read_excel(csv)[companies])

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
 
[WDM] - Driver [C:\Users\Lenovo\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


# Scraping [Google Patents](https://patents.google.com/) for Patents

## Sheet 1

## Deleting old csv files in directories (if any)

In [11]:
def delete(directory):
    filelist = [ f for f in os.listdir(directory) if f.endswith(".csv")]
    for f in filelist:
        os.remove(os.path.join(directory, f))

In [12]:
# from "Downloads" directory
delete('C:/Users/Lenovo/Downloads')

In [13]:
# from directory created
directory='C:/Users/Lenovo/Desktop/python scrap/'+str(today)
delete(directory)

In [14]:
# from subdirectory
delete(directory+'/Patent files')

## Scraping patents for each company

In [15]:
for i in l:
    try:
        link="https://patents.google.com/xhr/query?url=assignee%3D{}%26oq%3D{}&exp=&download=true".format("%2B".join(i.split()),"%2B".join(i.split()))
        # download csv
        webbrowser.open(link)
        # wait for 20 sec
        time.sleep(20)
        # read csv from downloads
        all_files = glob.glob(os.path.join(r'C:\Users\Lenovo\Downloads', "*.csv"))
        # rename csv
        os.rename(all_files[0],'C:/Users/Lenovo/Downloads/'+i+'.csv')
        # copy csv to req. folder
        shutil.copy('C:/Users/Lenovo/Downloads/'+i+'.csv', directory+'/Patent files')
        # delete csv from downloads
        delete('C:/Users/Lenovo/Downloads')
    except:
        pass

## Copying & Combining all output csv files

In [1]:
# read all csv files in subdirectory "Patent files"
all_files = glob.glob(os.path.join(directory+'/Patent files', "*.csv"))  
# concatenate all csv files present in "Patent files"
df_from_each_file = (pd.read_csv(f,header=1) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)
concatenated_df=concatenated_df[['id','title','publication date','assignee','inventor/author']]

NameError: name 'glob' is not defined

In [ ]:
concatenated_df.head()

In [ ]:
# filtering output based on "assignee"
appended_data = []
for company in l:
    s = pd.Series([str(i).lower() for i in concatenated_df['assignee']])
    k=s[s.str.contains(company.lower())].index
    if(list(k)!=[]):
        data = concatenated_df.loc[k]
        data['assignee']=[company for i in range(data.shape[0])]
        appended_data.append(data)
appended_data = pd.concat(appended_data)

In [ ]:
# Feature Engineering "Year Filed" from "publication date"
appended_data['Year Filed']=appended_data['publication date'].apply(lambda x:str(x).split('-')[0])
# selecting required columns
appended_data=appended_data[['assignee','id', 'title', 'publication date','Year Filed','inventor/author']]
# renaming columns
appended_data.columns=['Company Name/Assignee','Patent No.','Title','Publication Date','Year Filed','Inventors/Authors']
appended_data['Year Filed']=appended_data['Year Filed'].replace('nan',np.nan)

In [ ]:
appended_data.head()

In [ ]:
appended_data.shape[0]

## Filter Scraped Patents using [USPTO](http://patft.uspto.gov/)

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # open Chrome driver/window
links=[]
for i in l:
    term="+".join(i.split())
    driver.get('http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=0&f=S&l=50&TERM1={}&FIELD1=ASNM&co1=AND&TERM2=&FIELD2=&d=PTXT'.format(term))
    source = driver.page_source
    soup=bs4.BeautifulSoup(source, 'html.parser')
    #################### GET LINKS OF PAGES FOR EACH COMPANY #####################
    try:
        no_of_pages=int(soup.find_all('strong')[len(soup.find_all('strong'))-1].text)
        for page in range(no_of_pages):
            link='http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r={}&f=G&l=50&co1=AND&d=PTXT&s1=%22{}%22.ASNM.&OS=AN/%22{}al%22&RS=AN/%22{}%22'.format(page+1,term,term,term)
            links.append(link)
    except:
        pass

In [ ]:
title=[]
for i,j in enumerate(links):
    print(i)
    driver.get(j)
    source = driver.page_source
    soup=bs4.BeautifulSoup(source, 'html.parser')
    try:
        text=soup.find('font',{'size':'+1'}).text.strip()
        print(text)
        title.append(text)
    except:
        pass
    time.sleep(5)

In [ ]:
appended_data['Title']=appended_data['Title'].apply(lambda x:x.strip())
appended_data=appended_data.loc[appended_data['Title'].isin(list(set(title)))]

In [ ]:
appended_data=appended_data.drop_duplicates(subset=['Company Name/Assignee', 'Title'])
driver.close()

# Scraping [Clinical Trials](https://clinicaltrials.gov/)

In [ ]:
c=0
for i in l:
    try:
        link="https://clinicaltrials.gov/ct2/results/download_fields?cond=&term={}&type=&down_fmt=csv&down_flds=all&rslt=&age_v=&gndr=&intr=&titles=&outc=&spons=&lead=&id=&cntry=&state=&city=&dist=&locn=&fund=2&rsub=&strd_s=&strd_e=&prcd_s=&prcd_e=&sfpd_s=&sfpd_e=&rfpd_s=&rfpd_e=&lupd_s=&lupd_e=&sort=".format("+".join(i.split()))
        df = pd.read_csv(link)
        df=df[['First Posted','NCT Number','Title','Start Date','Completion Date','Sponsor/Collaborators']]
        df['Company Name']=[i for j in range(df.shape[0])]
        df.to_csv(directory+'/Trials files/'+i+'.csv',index=False)
    except:
        c=c+1
        #print(c,i+'.csv','not scraped')
        pass

In [ ]:
all_files = glob.glob(os.path.join(directory+'/Trials files', "*.csv"))  
if(all_files!=[]):
    a=pd.read_csv(all_files[0])
    for i in all_files[1:]:
        a=pd.concat([a,pd.read_csv(i)])
        
    a['Principal Investigator']=['-' for j in range(a.shape[0])]
    a=a[['Company Name','NCT Number','Title', 'Start Date', 'Completion Date','Sponsor/Collaborators','Principal Investigator']] 
    a.columns=['Company Name','Clinical Trial Identifier','Title of study',
               'Start Date','End Date','Sponsor Company','Principal Investigator']
    a['C'] = a.apply(lambda x: str(x['Company Name']) in str(x['Sponsor Company']), axis=1)
    a=a.loc[a['C']==True]
    a=a.drop('C',axis=1)

In [ ]:
if(all_files!=[]):
    ua = UserAgent()
    for i in range(a.shape[0]):
        url='https://clinicaltrials.gov/ct2/show/{}'.format(a.iloc[i,1])
        response = requests.get(url, {"User-Agent": ua.random})
        if response.status_code == 200:
            soup = bs4.BeautifulSoup(response.text, "html.parser")
            for j in soup.find_all('table',{'class':'ct-layout_table tr-indent2'}):
                m=j.text
                if('Study Director:' in m):
                    a.iloc[i,-1]=m[m.index('Study Director:')+16:]

# Specify Date Range

In [ ]:
try:
    d_range=input('Do you want for a date range(y/n): ')
    if(d_range=='y'):
        date1 = input('From(YYYY-MM-DD): ')
        date2 = input('To(YYYY-MM-DD): ')
        dates=[]
        dates2=[]
        start = datetime.datetime.strptime(date1, '%Y-%m-%d')
        end = datetime.datetime.strptime(date2, '%Y-%m-%d')
        step = datetime.timedelta(days=1)
    ################################################################
        while start <= end:
            dates.append (str(start.date()))
            start += step
        for i in dates:
            a1=i.split('-')
            x = datetime.datetime(int(a1[0]),int(a1[1]),int(a1[2]))
            dates2.append(x.strftime("%B %d, %Y"))
    ##################################################################
        appended_data=appended_data.loc[appended_data['Publication Date'].isin(dates)]
        a=a.loc[a['Start Date'].isin(dates2)]
except:
    pass

 # Adding not-scraped companies to DataFrame

In [ ]:
# patents
not_scraped=list(set(l)-set(appended_data['Company Name/Assignee'].unique()))
for i in not_scraped:
    s2 = pd.DataFrame([i,np.nan,np.nan,np.nan,np.nan,np.nan]).T
    s2.columns=appended_data.columns
    appended_data=appended_data.append(s2)

In [ ]:
# trials
if(all_files!=[]):
    not_scraped=list(set(l)-set(a['Company Name'].unique()))
    for i in not_scraped:
        s2 = pd.DataFrame([i,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]).T
        s2.columns=a.columns
        a=a.append(s2)

# Sheet 2

In [ ]:
# patents
appended_data2=appended_data.groupby(['Company Name/Assignee']).size().reset_index(name='Value')
appended_data2['Patent (Y/N)']=['Y' for i in range(appended_data2.shape[0])]
for i in range(appended_data2.shape[0]):
    if(appended_data2['Company Name/Assignee'][i] in not_scraped):
        appended_data2['Value'][i]=0
        appended_data2['Patent (Y/N)'][i]='N'

In [ ]:
# trials
appended_data3=a.groupby(['Company Name']).size().reset_index(name='Value')
appended_data3['Clinical Trials (Y/N)']=['Y' for i in range(appended_data3.shape[0])]
for i in range(appended_data3.shape[0]):
    if(appended_data3['Company Name'][i] in not_scraped):
        appended_data3['Value'][i]=0
        appended_data3['Clinical Trials (Y/N)'][i]='N'

# Save Outputs

In [ ]:
# Write Sheet 1 and Sheet 2 as excel
name='/BiosectRx Patent Harvest-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
#appended_data.reset_index(inplace=True)
#appended_data2.reset_index(inplace=True)
appended_data2.to_excel(writer, sheet_name='Patent Check',index=False)
appended_data.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()

In [ ]:
name='/BiosectRx Trials Harvest-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
a.reset_index(inplace=True)
appended_data3.reset_index(inplace=True)
appended_data3.to_excel(writer, sheet_name='Trials Check',index=False)
a.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()

# Populate in Template

In [ ]:
# merge patents and trials
df3=pd.merge(appended_data, a, left_on='Company Name/Assignee', right_on='Company Name')
df3=df3.drop(['Company Name'],axis=1)
df3=df3.drop_duplicates()

In [ ]:
# open original template to get format
df=pd.read_excel('Template.xlsx')
df.columns=df.iloc[0,:]
df.columns

In [ ]:
df5=df3.copy()
df5=df5[['Company Name/Assignee', 'Patent No.', 'Title', 'Year Filed',
       'Inventors/Authors', 'Clinical Trial Identifier',
       'Title of study', 'Start Date', 'End Date', 'Sponsor Company',
       'Principal Investigator']]

In [ ]:
# rename columns
df5.columns=['Company Name','Application No','IP Title','IP Year',
             'Inventors','Clinical Trial ID','Clinical Title',
            'Start Date', 'End Date', 'Sponsor', 'Investigator']

In [ ]:
df6=df5.iloc[:,:5]
dup_index=set(df6.index)-set(df6.drop_duplicates(keep='first').index)
df6.iloc[list(dup_index),1:]=np.NaN
df6

In [ ]:
df7=df5.iloc[:,5:]
df7['a']=df5['Company Name']
dup_index=set(df7.index)-set(df7.drop_duplicates(keep='first').index)
df7.iloc[list(dup_index)]=np.NaN
df7=df7.drop('a',axis=1)
df7

In [ ]:
x=pd.concat([df6,df7],axis=1)

In [ ]:
# add null columns as per template
for i in set(df.columns)-set(df5.columns):
    x[i]=np.nan
    
# rearrange columns as per template
x=x[df.columns]

In [ ]:
y=x.groupby('Company Name').count()[['Application No','Clinical Trial ID']]
l1,l2=[],[]
for i in y['Application No']:
    if(i==0):
        l1.append('N')
    else:
        l1.append('Y')
for i in y['Clinical Trial ID']:
    if(i==0):
        l2.append('N')
    else:
        l2.append('Y')

In [ ]:
y['Patent (Y/N)']=l1
y['Clinical Trials (Y/N)']=l2

In [ ]:
y=y[['Patent (Y/N)','Application No','Clinical Trials (Y/N)','Clinical Trial ID']]
y.columns=['Patent (Y/N)','Value','Clinical Trials (Y/N)','Value']

In [ ]:
y=y.sort_values(by=['Patent (Y/N)', 'Clinical Trials (Y/N)'])
ordered_classes = list(y.index)
df_list = []
for i in ordered_classes:
    df_list.append(x[x['Company Name']==i])
ordered_df = pd.concat(df_list)

In [ ]:
# save to excel
name='/BiosectRx TEMPLATE-'+str(today)+'.xlsx'
writer = pd.ExcelWriter(directory+name, engine='xlsxwriter')
y.to_excel(writer, sheet_name='Check',index=True)
x.to_excel(writer, sheet_name='Harvest',index=False)
writer.save()